In [1]:
import pandas as pd
import os
import glob
import time
import datetime

#### Combining All Loyalty Members' Transcation Data

In [2]:
file_list = os.listdir("/Users/bowenjin/Desktop/Lion's Choice/Data/loyalty transactions")

In [3]:
combined_data = pd.DataFrame()

In [4]:
for file_name in file_list:
    path = "/Users/bowenjin/Desktop/Lion's Choice/Data/loyalty transactions/" + file_name
    data = pd.read_csv(path, header = 1, low_memory=False)
    #print(len(data))
    #print(file_name)
    combined_data = pd.concat([combined_data,data])

In [5]:
combined_data

,Card Number,Account Code,Customer No.,Card Template,Transaction Date,Transaction Type,Store Merchant,Store Number,Store Name,Wallet Type,...,8 - Comp Dollars 8 - Balance,9 - Points 9 - Accrued,9 - Points 9 - Redeemed,9 - Points 9 - Balance,10 - Visits 10 - Accrued,10 - Visits 10 - Redeemed,10 - Visits 10 - Balance,0 - Stored Value 0 - Accrued,0 - Stored Value 0 - Redeemed,0 - Stored Value 0 - Balance
0,174100000622,32907372,NaN,Loyalty,2022-05-01 10:08,Campaign Adjustment,Lion's Choice,1,Ballwin,Rewards,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,174100001372,32907379,NaN,Loyalty,2022-05-01 03:08,Campaign Expiration,Lion's Choice,1,Ballwin,Rewards,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,174100001539,32907381,NaN,Loyalty,2022-05-01 03:08,Campaign Expiration,Lion's Choice,1,Ballwin,Rewards,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,174100001687,32907382,NaN,Loyalty,2022-05-01 03:08,Campaign Expiration,Lion's Choice,1,Ballwin,Rewards,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,174100002818,32907394,NaN,Loyalty,2022-05-01 10:08,Campaign Adjustment,Lion's Choice,1,Ballwin,Rewards,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3152,7560850044532419640,32851786,NaN,Incomm Schnucks,2022-04-10 12:32,Purchase Card,Lion's Choice,INCOMM,Incomm - Schnucks,Stored Value,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.00,10.00
3153,7560850044544953255,32851787,NaN,Incomm Schnucks,2022-04-10 14:13,Purchase Card,Lion's Choice,INCOMM,Incomm - Schnucks,Stored Value,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.00,10.00
3154,7560850048973207851,32852230,NaN,Incomm Schnucks,2022-04-10 13:02,Purchase Card,Lion's Choice,INCOMM,Incomm - Schnucks,Stored Value,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,0.00,20.00
3155,7571780007499841405,33119188,NaN,Migrated Gift Card,2022-04-10 13:16,Redeem Stored Value,Lion's Choice,13,Hanley Road,Stored Value,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-12.46,0.75


In [ ]:
#combined_data.to_csv('combined_data.csv')

####  Data Cleaning

In [34]:
all_columns = combined_data.columns.to_list()
numerical_columns = combined_data.describe().columns.to_list()
categorical_columns = []
for column in all_columns:
    if column not in numerical_columns:
        categorical_columns.append(column)

In [7]:
info_list = categorical_columns.copy()

for column in numerical_columns:
    if combined_data.describe().loc['count',column] != float(0):
        info_list.append(column)

In [8]:
info_list

['Customer No.',
 'Card Template',
 'Transaction Date',
 'Transaction Type',
 'Store Merchant',
 'Store Number',
 'Store Name',
 'Wallet Type',
 'Check No.',
 'Terminal ID',
 'Identification Method',
 'Account Status',
 'Sender',
 'Card Number',
 'Account Code',
 'Cashier ID',
 'Auth Code',
 '13 - $2 Off Butcher Block Sala 13 - Accrued',
 '13 - $2 Off Butcher Block Sala 13 - Redeemed',
 '13 - $2 Off Butcher Block Sala 13 - Balance',
 '1 - Dollars Spent 1 - Accrued',
 '1 - Dollars Spent 1 - Redeemed',
 '1 - Dollars Spent 1 - Balance',
 '2 - $3 Signup Reward 2 - Accrued',
 '2 - $3 Signup Reward 2 - Redeemed',
 '2 - $3 Signup Reward 2 - Balance',
 '3 - Reward Dollars 3 - Accrued',
 '3 - Reward Dollars 3 - Redeemed',
 '3 - Reward Dollars 3 - Balance',
 '11 - Free Regular Fry 11 - Accrued',
 '11 - Free Regular Fry 11 - Redeemed',
 '11 - Free Regular Fry 11 - Balance',
 '4 - Free Original Concrete 4 - Accrued',
 '4 - Free Original Concrete 4 - Redeemed',
 '4 - Free Original Concrete 4 - Bala

In [9]:
loyal_trans = combined_data.loc[:, info_list]

In [10]:
#make sure all transactions are from loyalty members
loyal_trans = loyal_trans[loyal_trans['Card Template'] == 'Loyalty']

In [11]:
#drop unuseful cols 
loyal_trans = loyal_trans.drop(columns = ['Store Merchant','Wallet Type','Terminal ID','Identification Method',
                                          'Account Status','Sender','Auth Code', '0 - Stored Value 0 - Accrued',
                                          '0 - Stored Value 0 - Redeemed', '0 - Stored Value 0 - Balance'])

In [12]:
loyal_trans['Transaction Time'] = loyal_trans['Transaction Date'].map(lambda x : x[-5:])
loyal_trans['Transaction Date'] = loyal_trans['Transaction Date'].map(lambda x : x[:-6])

loyal_trans = loyal_trans[loyal_trans['Transaction Date'] >= '2022-04-13']
loyal_trans = loyal_trans.reset_index()

In [13]:
loyal_trans

,index,Customer No.,Card Template,Transaction Date,Transaction Type,Store Number,Store Name,Check No.,Card Number,Account Code,...,5 - $2 Off Soup Bowl 5 - Accrued,5 - $2 Off Soup Bowl 5 - Redeemed,5 - $2 Off Soup Bowl 5 - Balance,9 - Points 9 - Accrued,9 - Points 9 - Redeemed,9 - Points 9 - Balance,10 - Visits 10 - Accrued,10 - Visits 10 - Redeemed,10 - Visits 10 - Balance,Transaction Time
0,0,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100000622,32907372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:08
1,1,NaN,Loyalty,2022-05-01,Campaign Expiration,1,Ballwin,web,174100001372,32907379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:08
2,2,NaN,Loyalty,2022-05-01,Campaign Expiration,1,Ballwin,web,174100001539,32907381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:08
3,3,NaN,Loyalty,2022-05-01,Campaign Expiration,1,Ballwin,web,174100001687,32907382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:08
4,4,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100002818,32907394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267716,5755,NaN,Loyalty,2022-05-13,Register,pxweb,Web Site,Guest Website,741019473031,49046645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:03
267717,5756,NaN,Loyalty,2022-05-13,Activate,5,South County,iOS,741019474963,49046649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:03
267718,5757,NaN,Loyalty,2022-05-13,Register,pxweb,Web Site,iOS,741019474963,49046649,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:03
267719,5758,NaN,Loyalty,2022-05-13,Activate,4,St. Peters,iOS,741019475143,49046662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00:36


In [14]:
test_data = loyal_trans[(loyal_trans['Transaction Type'] == 'Campaign Adjustment')&(loyal_trans['13 - $2 Off Butcher Block Sala 13 - Accrued'] > 0)]
test_data

,index,Customer No.,Card Template,Transaction Date,Transaction Type,Store Number,Store Name,Check No.,Card Number,Account Code,...,5 - $2 Off Soup Bowl 5 - Accrued,5 - $2 Off Soup Bowl 5 - Redeemed,5 - $2 Off Soup Bowl 5 - Balance,9 - Points 9 - Accrued,9 - Points 9 - Redeemed,9 - Points 9 - Balance,10 - Visits 10 - Accrued,10 - Visits 10 - Redeemed,10 - Visits 10 - Balance,Transaction Time
101932,0,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,174100000028,32907366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50
101933,1,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,174100000176,32907367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50
101934,2,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,174100000440,32907370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50
101935,3,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,174100000515,32907371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50
101936,4,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,174100000770,32907373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130991,29095,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,741018244425,48870478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50
130992,29096,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,741018245224,48870525,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50
130993,29097,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,741018246396,48870635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50
130994,29098,NaN,Loyalty,2022-04-13,Campaign Adjustment,corp,Corporate,web,741018247196,48870678,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:50


#### Cleaning Data For RFM Analysis

In [15]:
#get all loyalty members' card number, without duplicates
member_list = list(test_data['Card Number'].unique())
member_list

[174100000028,
 174100000176,
 174100000440,
 174100000515,
 174100000770,
 174100000903,
 174100001000,
 174100001166,
 174100001216,
 174100001687,
 174100001943,
 174100002123,
 174100002248,
 174100002313,
 174100002446,
 174100002586,
 174100002701,
 174100002818,
 174100002917,
 174100003030,
 174100003246,
 174100003329,
 174100003410,
 174100003600,
 174100003758,
 174100004244,
 174100004442,
 174100004566,
 174100004624,
 174100004772,
 174100004889,
 174100004988,
 174100005134,
 174100005209,
 174100005514,
 174100005647,
 174100005803,
 174100006074,
 174100006173,
 174100006223,
 174100006637,
 174100006728,
 174100006900,
 174100007056,
 174100007262,
 174100007312,
 174100007601,
 174100007973,
 174100008328,
 174100008476,
 174100008583,
 174100008815,
 174100008914,
 174100009268,
 174100009672,
 174100009748,
 174100009813,
 174100009953,
 174100010274,
 174100010456,
 174100011058,
 174100011280,
 174100011520,
 174100011876,
 174100012031,
 174100012429,
 174100012

In [16]:
partial_data = loyal_trans[loyal_trans['Card Number'].isin(member_list)]

In [17]:
partial_data

,index,Customer No.,Card Template,Transaction Date,Transaction Type,Store Number,Store Name,Check No.,Card Number,Account Code,...,5 - $2 Off Soup Bowl 5 - Accrued,5 - $2 Off Soup Bowl 5 - Redeemed,5 - $2 Off Soup Bowl 5 - Balance,9 - Points 9 - Accrued,9 - Points 9 - Redeemed,9 - Points 9 - Balance,10 - Visits 10 - Accrued,10 - Visits 10 - Redeemed,10 - Visits 10 - Balance,Transaction Time
3,3,NaN,Loyalty,2022-05-01,Campaign Expiration,1,Ballwin,web,174100001687,32907382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:08
4,4,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100002818,32907394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:08
5,5,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100002917,32907395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:08
6,6,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100003030,32907396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:08
7,7,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100003410,32907400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267290,5329,NaN,Loyalty,2022-05-12,Identify Customer,21,Ellisville,627d6df708730d0af90d7462,741018241694,48870311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16:30
267291,5330,NaN,Loyalty,2022-05-12,Identify Customer,21,Ellisville,627d6df708730d0af90d7462,741018241694,48870311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16:30
267292,5331,NaN,Loyalty,2022-05-12,Balance Inquiry,21,Ellisville,627d6df708730d0af90d7462,741018241694,48870311,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16:37
267293,5332,NaN,Loyalty,2022-05-12,Accrual / Redemption,21,Ellisville,627d6df708730d0af90d7462,741018241694,48870311,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,16:37


In [18]:
#Fill in all the NaN for the salad coupon
partial_data['13 - $2 Off Butcher Block Sala 13 - Redeemed'] = partial_data['13 - $2 Off Butcher Block Sala 13 - Redeemed'].map(lambda x : abs(x) if x < 0 else 0)

<ipython-input-18-db35e5a12b49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partial_data['13 - $2 Off Butcher Block Sala 13 - Redeemed'] = partial_data['13 - $2 Off Butcher Block Sala 13 - Redeemed'].map(lambda x : abs(x) if x < 0 else 0)


In [19]:
partial_data[(partial_data['Transaction Type'] == 'Campaign Expiration')
             &(partial_data['13 - $2 Off Butcher Block Sala 13 - Redeemed'] > 0)]

,index,Customer No.,Card Template,Transaction Date,Transaction Type,Store Number,Store Name,Check No.,Card Number,Account Code,...,5 - $2 Off Soup Bowl 5 - Accrued,5 - $2 Off Soup Bowl 5 - Redeemed,5 - $2 Off Soup Bowl 5 - Balance,9 - Points 9 - Accrued,9 - Points 9 - Redeemed,9 - Points 9 - Balance,10 - Visits 10 - Accrued,10 - Visits 10 - Redeemed,10 - Visits 10 - Balance,Transaction Time
138853,0,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,174100000028,32907366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05
138854,1,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,174100000176,32907367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05
138855,2,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,174100000440,32907370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05
138856,3,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,174100000515,32907371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05
138857,4,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,174100000770,32907373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167746,28999,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,741018244425,48870478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05
167747,29000,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,741018245224,48870525,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05
167748,29001,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,741018246396,48870635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05
167749,29002,NaN,Loyalty,2022-04-18,Campaign Expiration,corp,Corporate,web,741018247196,48870678,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03:05


In [20]:
#further sort out unuseful data based on transaction type
partial_data = partial_data[partial_data['Transaction Type'] != 'Campaign Expiration']

In [21]:
partial_data.iloc[:,:15]

,index,Customer No.,Card Template,Transaction Date,Transaction Type,Store Number,Store Name,Check No.,Card Number,Account Code,Cashier ID,13 - $2 Off Butcher Block Sala 13 - Accrued,13 - $2 Off Butcher Block Sala 13 - Redeemed,13 - $2 Off Butcher Block Sala 13 - Balance,1 - Dollars Spent 1 - Accrued
4,4,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100002818,32907394,32028502,NaN,0.0,NaN,NaN
5,5,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100002917,32907395,32028502,NaN,0.0,NaN,NaN
6,6,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100003030,32907396,32028502,NaN,0.0,NaN,NaN
7,7,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100003410,32907400,32028502,NaN,0.0,NaN,NaN
8,8,NaN,Loyalty,2022-05-01,Campaign Adjustment,1,Ballwin,web,174100003600,32907402,32028502,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267290,5329,NaN,Loyalty,2022-05-12,Identify Customer,21,Ellisville,627d6df708730d0af90d7462,741018241694,48870311,0,NaN,0.0,NaN,NaN
267291,5330,NaN,Loyalty,2022-05-12,Identify Customer,21,Ellisville,627d6df708730d0af90d7462,741018241694,48870311,0,NaN,0.0,NaN,NaN
267292,5331,NaN,Loyalty,2022-05-12,Balance Inquiry,21,Ellisville,627d6df708730d0af90d7462,741018241694,48870311,0,NaN,0.0,NaN,NaN
267293,5332,NaN,Loyalty,2022-05-12,Accrual / Redemption,21,Ellisville,627d6df708730d0af90d7462,741018241694,48870311,0,NaN,0.0,NaN,NaN


In [22]:
#select columns uselful for RFM analysis
rfm_data = partial_data.loc[:, ['Card Number','1 - Dollars Spent 1 - Balance','10 - Visits 10 - Balance',
                               '13 - $2 Off Butcher Block Sala 13 - Redeemed']]
rfm_data

,Card Number,1 - Dollars Spent 1 - Balance,10 - Visits 10 - Balance,13 - $2 Off Butcher Block Sala 13 - Redeemed
4,174100002818,NaN,NaN,0.0
5,174100002917,NaN,NaN,0.0
6,174100003030,NaN,NaN,0.0
7,174100003410,NaN,NaN,0.0
8,174100003600,NaN,NaN,0.0
...,...,...,...,...
267290,741018241694,NaN,NaN,0.0
267291,741018241694,NaN,NaN,0.0
267292,741018241694,NaN,NaN,0.0
267293,741018241694,NaN,1.0,0.0


In [23]:
rfm_trans = rfm_data.groupby(['Card Number']).max()
rfm_trans

,1 - Dollars Spent 1 - Balance,10 - Visits 10 - Balance,13 - $2 Off Butcher Block Sala 13 - Redeemed
Card Number,,,
174100000028,195.66,15.0,0.0
174100000176,562.74,24.0,0.0
174100000440,NaN,NaN,0.0
174100000515,164.85,13.0,0.0
174100000770,NaN,NaN,0.0
...,...,...,...
741018244425,29.48,3.0,0.0
741018245224,NaN,NaN,0.0
741018246396,NaN,NaN,0.0


In [24]:
#read in loyalty account info data
accounts_data = pd.read_csv("/Users/bowenjin/Desktop/Lion's Choice/Data/loyal_info.csv")
accounts_data.head()

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,14,18,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Email Opt-In,Push Opt-In,SMS Opt-In,SMS Opt-In Verified,Email Failed,SanitizedEmail,Is Registered,Title,First Name,Last Name,...,Enroll Store Code,Enroll Store Name,Favorite Store Code,Favorite Store Name,Register Date,Last Guest Activity Date,Phone Area Code,Mobile Phone Area Code,Card Number,Account Code
0,No,Yes,No,No,NaN,****@paytronix.com,Yes,NaN,Dave,Removed,...,pxweb,Web Site,NaN,NaN,1/25/2021,1/19/2022,303.0,303.0,741000000835,32013214
1,No,No,No,No,NaN,****@paytronix.com,Yes,NaN,NaN,Removed,...,SMS,Loyalty SMS,NaN,NaN,1/25/2021,1/25/2021,201.0,201.0,741000001320,32013323
2,Yes,No,No,No,Yes,****@paytronix.com,Yes,NaN,Test,Removed,...,SMS,Loyalty SMS,NaN,NaN,1/25/2021,1/25/2021,914.0,914.0,741000002765,32013354
3,Yes,No,Yes,Yes,NaN,****@paytronix.com,Yes,NaN,NaN,Removed,...,SMS,Loyalty SMS,NaN,NaN,2/3/2021,2/3/2021,305.0,305.0,741000003540,32030188
4,Yes,No,Yes,Yes,NaN,****@paytronix.com,Yes,NaN,NaN,Removed,...,SMS,Loyalty SMS,NaN,NaN,2/3/2021,2/3/2021,617.0,617.0,741000004712,32030190


In [25]:
#sort the info data based on the unique cardnumber and get only the last activity date col
partial_accounts = accounts_data[accounts_data['Card Number'].isin(member_list)].loc[:, ['Card Number', 'Last Guest Activity Date']]

In [26]:
partial_accounts

,Card Number,Last Guest Activity Date
0,741000000835,1/19/2022
1,741000001320,1/25/2021
2,741000002765,1/25/2021
3,741000003540,2/3/2021
4,741000004712,2/3/2021
...,...,...
71675,741018244425,5/26/2022
71676,741018245224,4/12/2022
71677,741018246396,4/12/2022
71678,741018247196,5/10/2022


In [27]:
#merge the two dataset on card number
rfm_trans = rfm_trans.merge(partial_accounts, on = 'Card Number')
rfm_trans

,Card Number,1 - Dollars Spent 1 - Balance,10 - Visits 10 - Balance,13 - $2 Off Butcher Block Sala 13 - Redeemed,Last Guest Activity Date
0,174100000028,195.66,15.0,0.0,4/26/2022
1,174100000176,562.74,24.0,0.0,5/5/2022
2,174100000440,NaN,NaN,0.0,2/26/2022
3,174100000515,164.85,13.0,0.0,5/3/2022
4,174100000770,NaN,NaN,0.0,3/25/2022
...,...,...,...,...,...
25758,741018244425,29.48,3.0,0.0,5/26/2022
25759,741018245224,NaN,NaN,0.0,4/12/2022
25760,741018246396,NaN,NaN,0.0,4/12/2022
25761,741018247196,33.25,3.0,0.0,5/10/2022


In [28]:
#convert NaN to 0
rfm_trans['1 - Dollars Spent 1 - Balance'] = rfm_trans['1 - Dollars Spent 1 - Balance'].map(lambda x : x if x > 0 else 0)
rfm_trans['10 - Visits 10 - Balance'] = rfm_trans['10 - Visits 10 - Balance'].map(lambda x : x if x > 0 else 0)

In [29]:
#calculate the number of days since the last activity
rfm_trans['Last Guest Activity Date'] = rfm_trans['Last Guest Activity Date'].map(lambda x: time.strftime('%Y-%m-%d', time.strptime(x, '%m/%d/%Y'))) 
rfm_trans['Last Guest Activity Date'] = rfm_trans['Last Guest Activity Date'].map(lambda x: (datetime.datetime.strptime('2022-05-27', '%Y-%m-%d') - datetime.datetime.strptime(x, '%Y-%m-%d')).days) 

In [30]:
#final cleaned data for rfm analysis
rfm_trans

,Card Number,1 - Dollars Spent 1 - Balance,10 - Visits 10 - Balance,13 - $2 Off Butcher Block Sala 13 - Redeemed,Last Guest Activity Date
0,174100000028,195.66,15.0,0.0,31
1,174100000176,562.74,24.0,0.0,22
2,174100000440,0.00,0.0,0.0,90
3,174100000515,164.85,13.0,0.0,24
4,174100000770,0.00,0.0,0.0,63
...,...,...,...,...,...
25758,741018244425,29.48,3.0,0.0,1
25759,741018245224,0.00,0.0,0.0,45
25760,741018246396,0.00,0.0,0.0,45
25761,741018247196,33.25,3.0,0.0,17


In [31]:
rfm_trans.describe()

,Card Number,1 - Dollars Spent 1 - Balance,10 - Visits 10 - Balance,13 - $2 Off Butcher Block Sala 13 - Redeemed,Last Guest Activity Date
count,2.576300e+04,25763.000000,25763.000000,25763.000000,25763.000000
mean,4.300835e+11,140.370659,11.727866,0.004930,63.655009
std,2.821253e+11,308.733951,28.893380,0.070039,58.131860
min,1.741000e+11,0.000000,0.000000,0.000000,0.000000
25%,1.741018e+11,0.000000,0.000000,0.000000,14.000000
50%,1.741041e+11,0.000000,0.000000,0.000000,48.000000
75%,7.410112e+11,161.860000,13.000000,0.000000,104.000000
max,7.410182e+11,6510.820000,954.000000,1.000000,1656.000000


In [43]:
#rfm_trans.to_csv('rfm_trans.csv')

#### Combining POS transaction data and some basic cleaning

In [35]:
#Combining all files
files_joined = os.path.join("/Users/bowenjin/Desktop/Lion's Choice/Data/pos transaction detail", "OC*")
list_files = glob.glob(files_joined)
df_pos_combined = pd.concat(map(pd.read_csv, list_files), ignore_index=True)

In [36]:
df_pos_combined.head()

,Location,Closed Date/Time,Employee Name,Item ID,Item Name,Item PLU,Price,Discount Total,Promotion Total,Taxes,Net Sales,Gross Sales,Total Sales,Revenue Center,Has Employee Discount,Destination,Voided,Has Customer,Is Modifier,Order ID
0,LC Corporate ***Office - Lab***,4/22/2022 1:30 AM,None,640246517.0,Notes,NaN,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,Condiments,0.0,PX Pickup,0.0,0.0,0.0,3.112384e+15
1,LC Corporate ***Office - Lab***,4/22/2022 1:30 AM,None,640247487.0,Regular Strawberry Shake (Online),NaN,$2.00,$0.00,$0.00,$0.18,$2.00,$2.00,$2.18,Desserts,0.0,PX Pickup,0.0,0.0,0.0,3.112384e+15
2,LC Corporate ***Office - Lab***,4/22/2022 1:30 AM,None,640237610.0,OLO Only Regular Concrete,NaN,$2.00,$0.00,$0.00,$0.19,$2.00,$2.00,$2.19,Desserts,0.0,PX Pickup,0.0,0.0,0.0,3.112384e+15
3,LC Corporate ***Office - Lab***,4/22/2022 1:30 AM,None,640214520.0,Strawberry (r),NaN,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,Condiments,0.0,PX Pickup,0.0,0.0,1.0,3.112384e+15
4,LC Corporate ***Office - Lab***,4/22/2022 1:30 AM,None,640242676.0,Spoon,NaN,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,Condiments,0.0,PX Pickup,0.0,0.0,1.0,3.112384e+15


In [37]:
df_pos_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4037135 entries, 0 to 4037134
Data columns (total 20 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Location               object 
 1   Closed Date/Time       object 
 2   Employee Name          object 
 3   Item ID                float64
 4   Item Name              object 
 5   Item PLU               float64
 6   Price                  object 
 7   Discount Total         object 
 8   Promotion Total        object 
 9   Taxes                  object 
 10  Net Sales              object 
 11  Gross Sales            object 
 12  Total Sales            object 
 13  Revenue Center         object 
 14  Has Employee Discount  float64
 15  Destination            object 
 16  Voided                 float64
 17  Has Customer           float64
 18  Is Modifier            float64
 19  Order ID               float64
dtypes: float64(7), object(13)
memory usage: 616.0+ MB


In [ ]:
#Data Exploration and Cleaning for building Tableau dashborad

In [38]:
df_pos_combined['Voided'].unique()

array([ 0.,  1., nan])

In [39]:
df_pos_combined['Item Name'].unique()

array(['Notes', 'Regular Strawberry Shake (Online)',
       'OLO Only Regular Concrete', 'Strawberry (r)', 'Spoon',
       'Strawberry Sundae', 'Regular Turkey (pp) (px)', 'Provel Cheese',
       'Seasoning', 'Regular Ham (pp) (px)', 'Swiss Cheese',
       'Regular Roast Beef (pp) (px)', 'Cheddar Cheese', 'GF Bun',
       'Spicy Giard', 'Cookie Platter (Cater)', 'Asst. Cookie (Cater)',
       'Choc Chip Cookie (Cater) ', 'Sugar Cookie (Cater) ',
       'Regular Ham (c) (Cater) ', 'Regular Ham Meal (Cater)',
       'House Chips (Cater) ', 'King Roast Beef (c)', 'Well Done',
       'SEE CASHIER!', 'Large Fries', 'Large Drink',
       'Diet Dr. Pepper (l)', 'King Roast Beef Meal',
       'Regular Roast Beef', 'Bowl Broccoli Cheese Soup', 'Baked Potato',
       'Sour Cream', 'Butter', 'Broccoli', 'Cup Chili',
       'Seasoning (Packet)', 'Au Jus', 'Regular Roast Beef (c)',
       'Regular Fries', 'Reg Drink', 'Dr. Pepper (r)',
       'Regular Roast Beef Meal', 'Coke (r)', 'No Condiment', '

In [40]:
df_pos_combined = df_pos_combined.drop(columns = ['Item PLU','Has Employee Discount','Has Customer','Is Modifier'])

In [41]:
list(df_pos_combined)

['Location',
 'Closed Date/Time',
 'Employee Name',
 'Item ID',
 'Item Name',
 'Price',
 'Discount Total',
 'Promotion Total',
 'Taxes',
 'Net Sales',
 'Gross Sales',
 'Total Sales',
 'Revenue Center',
 'Destination',
 'Voided',
 'Order ID']

In [42]:
#df_pos_combined.to_csv('pos_combined.csv')